In [1]:
import numpy as np
import pandas as pd
import sys, os
import sqlalchemy as sql

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from draw_analysis.team_draw_analysis import TeamDrawAnalysis
from draw_analysis.functions import create_engine_msql, pyodbc_connection_and_cursor_msql

### 1. Data

In [2]:
raw_data = pd.read_csv('../1_extract_transform_analyse/backup/segunda_division_fixtures_2018_2019_2020_2021_07_13.csv')
segdiv = TeamDrawAnalysis(raw_data)
teams_id = segdiv.get_teams_id()

 Data description: 
	Country: Spain 
 	League name: Segunda Division 
 	League id: 141 
 	Seasons: 2018, 2019, 2020 



### 2. Ms sql credentials for sqlalchemy and pyodbc

In [3]:
conn_str = ('Driver={SQL Server Native Client 11.0};'
           'Server=DESKTOP-G6VJ78V;'
           'Trusted_Connection=yes;')

login = 'Daniel_SQL'
password = 'daniel123'
server = 'DESKTOP-G6VJ78V'
database = 'SegundaDivision'
driver = 'SQL+Server+Native+Client+11.0'

### 3. Create database and schema

- simpliefied database with team id relationship

In [4]:
conn, cursor = pyodbc_connection_and_cursor_msql(conn_str)
"""
cursor.execute ("DROP DATABASE SegundaDivision")
cursor.execute("CREATE DATABASE SegundaDivision")
cursor.execute("USE SegundaDivision")
cursor.execute("CREATE SCHEMA sd")"""

'\ncursor.execute ("DROP DATABASE SegundaDivision")\ncursor.execute("CREATE DATABASE SegundaDivision")\ncursor.execute("USE SegundaDivision")\ncursor.execute("CREATE SCHEMA sd")'

In [5]:
# sqlalchemy engine
engine = create_engine_msql(login, password, server, database, driver)


### 2. Adding fixtures table to db

In [6]:
fixtures = segdiv.fixtures_data
#fixtures.set_index('fixture_id', inplace=True)
#fixtures.to_sql('fixtures', engine, 'sd',if_exists='append', index_label='fixture_id', dtype={'fixture_date': sql.DateTime})

### 3. Adding draw/no draw series

In [7]:
# no draw series
no_draw = segdiv.teams_draw_series(series_type=0)
no_draw.reset_index(inplace=True)
#no_draw.to_sql('no_draw_series', engine, 'sd', if_exists='replace')

In [8]:
# draw series
draw_series = segdiv.teams_draw_series(series_type=1)
draw_series.reset_index(inplace=True)
draw_series['team_name'] = draw_series['team_name'].map(teams_id)
draw_series[['team_name', 'league_season', 'draw_series', 'from_to_date']] = draw_series[['team_name', 'league_season', 'draw_series', 'from_to_date']].astype('str')

In [9]:
"""
for i in range(len(draw_series)):
    cursor.execute('''
                    INSERT INTO SegundaDivision.sd.draw_series (team_id, league_season, draw_series, from_to_date)
                    VALUES (?, ?, ?, ?)''',tuple(draw_series.iloc[i]))
"""

"\nfor i in range(len(draw_series)):\n    cursor.execute('''\n                    INSERT INTO SegundaDivision.sd.draw_series (team_id, league_season, draw_series, from_to_date)\n                    VALUES (?, ?, ?, ?)''',tuple(draw_series.iloc[i]))\n"

### 4. League tables

In [10]:
df_list =[]

sd_seasons = segdiv.seasons
for season in sd_seasons:
    league_table = segdiv.league_table(season=season)
    league_table['season'] = season
    league_table['Team'] = league_table['Team'].map(teams_id)
    league_table.reset_index(inplace=True)
    df_list.append(league_table)
    

concat_league_tables = pd.concat(df_list)
concat_league_tables.astype('str')

,index,Team,MP,W,D,L,GF,GA,GD,Pts,season
0,1,727,42,26,9,7,59,35,24,87,2018
1,2,715,42,22,13,7,52,28,24,79,2018
2,3,535,42,21,11,10,51,31,20,74,2018
3,4,722,42,19,14,9,54,38,16,71,2018
4,5,798,42,19,12,11,53,37,16,69,2018
...,...,...,...,...,...,...,...,...,...,...,...
17,18,716,42,11,14,17,38,53,-15,47,2020
18,19,9593,42,11,13,18,40,48,-8,46,2020
19,20,5280,42,11,11,20,28,53,-25,44,2020
20,21,5254,42,11,8,23,35,54,-19,41,2020


In [11]:
"""
for i in range(len(concat_league_tables)):
    cursor.execute('''
                    INSERT INTO SegundaDivision.sd.league_table (table_pos, team_id, MP, W, D, L, GF, GA, GD, Pts, season)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',tuple(concat_league_tables.iloc[i]))
"""

"\nfor i in range(len(concat_league_tables)):\n    cursor.execute('''\n                    INSERT INTO SegundaDivision.sd.league_table (table_pos, team_id, MP, W, D, L, GF, GA, GD, Pts, season)\n                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',tuple(concat_league_tables.iloc[i]))\n"

### 5. Team draw performance

In [45]:
draw_perf_df = []

for season in sd_seasons:
    draw_performance = segdiv.draw_results_performance(seasons=season, transpose=True)
    draw_performance.reset_index(inplace=True)
    draw_performance['team_name'] = draw_performance['team_name'].map(teams_id)
    draw_perf_df.append(draw_performance)
    
draw_perf_concat = pd.concat(draw_perf_df)
draw_perf_concat = draw_perf_concat.astype('str')

In [46]:
"""
for i in range(len(draw_perf_concat)):
    cursor.execute('''
                    INSERT INTO SegundaDivision.sd.draw_performance (team_id, season, table_pos, draws_pcts, 
                    no_draw_max, no_draw_mean, draw_max, draw_mean, goals_mean, goals_diff)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',tuple(draw_perf_concat.iloc[i]))
"""